In [1]:
import sys
print(sys.executable)

/opt/miniconda3/bin/python


In [1]:
# Setting up the environment

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.discovery import build
import pandas as pd

In [2]:
# Configuration

SERVICE_ACCOUNT_FILE = 'vidhya-etl-dabd5ec76159.json'  # Path to your downloaded JSON file
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
          'https://www.googleapis.com/auth/drive']  # Read & write scope
SPREADSHEET_ID = '1Nd2HsHCdqV7tRe4PN2rkB0Bg7-Bpv5vlRx4QpJEyc-A'  # From the URL between /d/...../edit
RANGE_NAME = 'Sheet1'  # Adjust as needed

In [3]:
# Authenticate

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes = SCOPES)

sheets_service = build('sheets', 'v4', credentials = creds)
drive_service  = build('drive', 'v3', credentials = creds)

In [4]:
# Read Data

sheet = sheets_service.spreadsheets()
result = sheet.values().get(spreadsheetId = SPREADSHEET_ID,
                            range = RANGE_NAME).execute()
values = result.get('values', [])

In [5]:
# Convert to Dataframe

if not values:
    print("No data found.")
else:
    job_tracker = pd.DataFrame(values[1:], columns=values[0])  # Skip header row
    display(job_tracker.head())

,Application Number,Date,Status,Role,Company,Source,Application URL,Deadline,Notes
0,,25/05/06,Applied,Data Analyst,Meta Inc.,LinkedIn,https://linkedin.com,25/05/10,find connections next!
1,,25/05/07,Applied,Data Analyst,Meta Inc.,LinkedIn,https://linkedin.com,25/05/11,find connections next!
2,,25/05/08,Applied,Data Analyst,Meta Inc.,LinkedIn,https://linkedin.com,25/05/12,find connections next!
3,,25/05/09,Applied,Data Analyst,Meta Inc.,LinkedIn,https://linkedin.com,25/05/13,find connections next!
4,,25/05/10,Applied,Data Analyst,Meta Inc.,LinkedIn,https://linkedin.com,25/05/14,find connections next!


In [6]:
# Convert transformed dataframe to list of lists

values = [job_tracker.columns.to_list()] + job_tracker.values.tolist()

In [7]:
# Create a new sheet

spreadsheet_body = {
    'properties': {
        'title': 'vidhya-etl-SSOT'
    }
}
new_sheet = sheets_service.spreadsheets().create(body=spreadsheet_body).execute()
NEW_SPREADSHEET_ID = new_sheet['spreadsheetId']
print(f"Created new spreadsheet: https://docs.google.com/spreadsheets/d/{NEW_SPREADSHEET_ID}")

# Write to new sheet

sheets_service.spreadsheets().values().update(
    spreadsheetId = NEW_SPREADSHEET_ID,
    range = 'Sheet1!A1',
    valueInputOption = 'USER_ENTERED',
    body={'values': values}
).execute()

# Share new sheet to my self

drive_service.permissions().create(
    fileId=NEW_SPREADSHEET_ID,
    body={
        'type': 'user',
        'role': 'writer',  # or 'reader'
        'emailAddress': 'nadimkhn323@gmail.com'
    },
    sendNotificationEmail=True
).execute()

Created new spreadsheet: https://docs.google.com/spreadsheets/d/13d7ChEPFco_MTAxtYHxIDD0b5OJIBOb9hPBsuHYhJ2U


{'kind': 'drive#permission',
 'id': '16585187306654007841',
 'type': 'user',
 'role': 'writer'}

### Next Steps

- Figure out to write it to a new google sheet
- clone jupyter notebook & rename as prod
- Connect with Vidhya's google sheet & don't write